<a href="https://colab.research.google.com/github/Montse1708/Practica4_Dashboard/blob/main/TI_1_Practica4_JessicaMontserratMoralesEnrique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4

**Nombre:** Jessica Montserrat Morales Enrique  
**e-mail:** jessica.morales5556@alumnos.udg.mx

## MODULES

In [ ]:
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

##CLASSES

In [ ]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

##FUNCTIONS

###Brownian Motion (BM)

In [ ]:
###############################################################################################
# Brownian Motion Trajectory
###############################################################################################
def BM_2d(n_steps=1000, speed=6, s_x_pos=0, s_y_pos=0):
    """
    Arguments:
        n_steps:
        speed:
        s_pos:
    Returns:
        BM_2d_df:
    """
    # Init velocity vector
    velocity = Vec2d(speed,0)

    BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps-1):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    return BM_2d_df

In [ ]:
#Verificar que la función es correcta con el gráfico de la Brownian Motion

#Plot
BM_2d_df_2 = BM_2d(2000, 9, 5, 7)

#Init figure
fig_BM_2d = go.Figure()

#Plot trajectory
fig_BM_2d.add_trace(go.Scatter(
                      x = BM_2d_df_2.x_pos,
                      y = BM_2d_df_2.y_pos,
                      marker = dict(size=2),
                      line = dict(width=1),
                      mode='lines',
                      name='BM_2d_trajectory',
                      showlegend=True
))

#Actualizar el diseño para agregar el título
fig_BM_2d.update_layout(title='Trayectoria de Brownian Motion en 2D')

#Mostrar el gráfico
fig_BM_2d.show()

###Correlated Random Walk (CRW)

In [87]:
######################################################################
# Correlated Random Walks (CRW)
######################################################################
def CRW_2d(CRW_exponents=0.2, resolution=800, speed=6, x_pos=0, y_pos=0):
    # Genera una serie de ángulos aleatorios de la distribución wrapcauchy
    wrapcauchy_rvs = wrapcauchy.rvs(CRW_exponents, size=resolution)

    # Inicializa un DataFrame para almacenar las posiciones x e y
    CRW_2d_df_s = pd.DataFrame(columns=['x_pos', 'y_pos'])
    # Agrega la posición inicial al DataFrame
    temp_df = pd.DataFrame([{'x_pos': x_pos, 'y_pos': y_pos}])
    CRW_2d_df_s = pd.concat([CRW_2d_df_s, temp_df], ignore_index=True)

    # Inicializa un vector de velocidad con la dirección inicial hacia la derecha y la velocidad dada
    velocity = Vec2d(speed, 0)

    # Genera las posiciones para cada paso en la caminata
    for j in range(resolution):
        # Actualiza la dirección del vector de velocidad utilizando el ángulo aleatorio
        velocity = velocity.rotated(wrapcauchy_rvs[j])
        # Calcula la nueva posición sumando la posición anterior con la velocidad actual
        temp_df = pd.DataFrame([{'x_pos': CRW_2d_df_s.x_pos[j] + velocity.x,
                                 'y_pos': CRW_2d_df_s.y_pos[j] + velocity.y}])
        # Agrega la nueva posición al DataFrame
        CRW_2d_df_s = pd.concat([CRW_2d_df_s, temp_df], ignore_index=True)

    return CRW_2d_df_s

In [ ]:
# Verificar que la función es correcta con el gráfico de Correlated Random Walk (CRW)
CRW_df = CRW_2d(0.8, 500, 5, 5, 7)

# Inicializar el gráfico
fig_CRW = go.Figure()

# Agregar la trayectoria al gráfico
fig_CRW.add_trace(go.Scatter(
    x = CRW_df.x_pos,
    y = CRW_df.y_pos,
    marker = dict(size=2),
    line = dict(width=1),
    mode='lines',
    name='CRW_trajectory',
    showlegend=True
))

# Actualizar el diseño para agregar el título
fig_CRW.update_layout(title='Trayectoria de Correlated Random Walk (CRW) en 2D')

# Mostrar el gráfico
fig_CRW.show()

###Lévy Flight (LF)

In [96]:
######################################################################
# Lévy Flight (LF)
######################################################################
def LF_2d(Levy_exponent=0.7, miu=3.0, beta=0, n_steps=250, x_pos=0, y_pos=0):
    # Genera una serie de valores aleatorios de la distribución Levy stable
    Levy_rvs = levy_stable.rvs(Levy_exponent, beta, loc=miu, size=n_steps)

    # Genera una serie de ángulos aleatorios de la distribución wrapcauchy
    Wrapcauchy_rvs = wrapcauchy.rvs(c=Levy_exponent, loc=beta, size=n_steps)
    # Calcula la suma acumulativa de los ángulos
    cauchy_rvs = Wrapcauchy_rvs
    for n in range(len(cauchy_rvs)):
        for m in range(n):
            cauchy_rvs[n] += cauchy_rvs[m]

    # Calcula el valor absoluto de los valores de la distribución Levy
    Levy_rvs_abs = abs(Levy_rvs)

    # Inicializa un DataFrame para almacenar las posiciones x e y
    LF_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    # Agrega la posición inicial al DataFrame
    temp_df = pd.DataFrame([{'x_pos': x_pos, 'y_pos': y_pos}])
    LF_2d_df = pd.concat([LF_2d_df, temp_df], ignore_index=True)

    # Genera las posiciones para cada paso en la caminata
    for i in range(n_steps-1):
        # Inicializa un vector de velocidad con la magnitud dada por el valor absoluto de Levy_rvs y dirección aleatoria
        velocity = Vec2d(Levy_rvs_abs[i], 0)
        velocity = velocity.rotated(cauchy_rvs[i])

        # Calcula la nueva posición sumando la posición anterior con la velocidad actual
        temp_df = pd.DataFrame([{'x_pos': LF_2d_df.x_pos[i] + velocity.x,
                                 'y_pos': LF_2d_df.y_pos[i] + velocity.y}])
        # Agrega la nueva posición al DataFrame
        LF_2d_df = pd.concat([LF_2d_df, temp_df], ignore_index=True)

    return LF_2d_df

In [ ]:
# Verificamos que la función LF_2d funcione correctamente
LF_2d_df = LF_2d(0.8, 3.0, 0, 500, 1.0, 3.0)

# Creamos una figura de Plotly
fig_LF_2d = go.Figure()

# Agregamos la trayectoria de la caminata de Lévy a la figura
fig_LF_2d.add_trace(go.Scatter(
    x=LF_2d_df.x_pos,
    y=LF_2d_df.y_pos,
    marker=dict(size=2),
    line=dict(width=1),
    mode='lines',
    name='LF_2d',
    showlegend=True
))

# Actualizamos el diseño de la figura con un título
fig_LF_2d.update_layout(title_text='Levy flight in 3D')

# Mostramos la figura
fig_LF_2d.show()

##METRICAS

In [ ]:
def distance_euclidean(point1, point2):
    # Desempaqueta las coordenadas de los puntos en variables individuales
    x1, y1 = point1
    x2, y2 = point2
    # Calcula la distancia euclidiana entre los dos puntos utilizando la fórmula
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

###Path length

In [78]:
def path_length(trajectory=pd.DataFrame):
    # Calcula la distancia euclidiana entre cada par de puntos consecutivos en la trayectoria y guarda los resultados en un arreglo
    pl = np.array([distance_euclidean(trajectory.loc[i-1], trajectory.iloc[i]) for i in range(1, trajectory.shape[0])])
    # Calcula la suma acumulativa de las distancias para obtener la longitud total del recorrido
    return np.cumsum(pl)

###Mean Squared Displacement

In [105]:
def MSD(df = pd.DataFrame):
  msd_2 = np.empty(shape=(0))
  list_msd = []

  for i in range(1,df.shape[0]):
    disp_vec = np.array([distance_euclidean(df.iloc[n-i], df.iloc[n]) for n in range(i, df.shape[0],1)])
    list_msd.append(np.sum(np.power(disp_vec, 2))/len(disp_vec))

  msd_2 = np.asarray(list_msd)

  return msd_2

##DASHBOARD

In [108]:
###############################################################################################
# WIDGETS
###############################################################################################
pn.extension()
# Crear un panel con texto Markdown y un título
# Texto en formato markdown para el panel
markdown_text = """
# DASHBOARD

### Panel params
"""
# Crear un panel con el texto markdown
panel = pn.panel(markdown_text)

# Tamaño común para los controles
size = 200
# Grupo de botones de radio para seleccionar entre BM, CRW, LF
option = pnw.RadioButtonGroup(name='option', width=size, options=['BM','CRW','LF'])
# Control deslizante para ajustar el número de pasos en la caminata
n_steps = pnw.IntSlider(name='Number of steps', width=size, value=0, step=20, start=0, end=1000)
# Control deslizante para ajustar un coeficiente
cauchy = pnw.FloatSlider(name='Coeficent', width=size, value=0.02, step=0.01, start=0.02, end=0.99)
# Control deslizante para ajustar el parámetro beta
alpha = pnw.IntSlider(name='alpha', width=size, value=0, step=1, start=-1, end=1)
# Controles de entrada numérica para ajustar las posiciones inicial en x
x_pos = pnw.IntInput(name='X position', width=90, value=0, step=1, start=0, end=25)
# Controles de entrada numérica para ajustar las posiciones inicial en y
y_pos = pnw.IntInput(name='Y position', width=90, value=0, step=1, start=0, end=25)
# Control deslizante para ajustar la velocidad
speed = pnw.IntSlider(name='Speed', width=size, value=0, step=1, start=0, end=20)
# Menú desplegable para seleccionar entre dos métricas: Path Length y MSD
metric_type = pnw.Select(name='Metric', width=size, value='Path Length', options=['Path Length','MSD'])



###############################################################################################
# FUNCIONES DEL DASHBOARD
###############################################################################################

#Función que toma el widget de acuerdo al tipo de trayectoria elegido
@pn.depends(option)
def opc(option_value):
    if option_value == 'BM':
        return pn.Column(n_steps, pn.Row(x_pos, y_pos), speed, metric_type)
    elif option_value == 'CRW':
        return pn.Column(n_steps, pn.Row(x_pos, y_pos), speed, cauchy, metric_type)
    elif option_value == 'LF':
        return pn.Column(n_steps, pn.Row(x_pos, y_pos), cauchy, alpha, metric_type)

#Función para el plot del tipo de trayectoria elegida
@pn.depends(option)
def plot_trajectory(option):
  if option == 'BM':
    return BW
  elif option == 'CRW':
    return CRW
  elif option == 'LF':
    return LF

#Función para el plot de Brownian Motion
@pn.depends(n_steps, x_pos, y_pos, speed)
def BW(n_steps, x_pos, y_pos, speed):
    bm_df_1 = BM_2d(n_steps, speed, x_pos, y_pos)
    fig_BM = go.Figure()
    fig_BM.add_trace(go.Scatter3d(
        x=bm_df_1.x_pos,
        y=bm_df_1.y_pos,
        z=np.linspace(0, 1, n_steps),
        marker=dict(size=2),
        line=dict(width=2),
        mode='lines',
        name='BM',
        showlegend=True
    ))

    fig_BM.update_layout(title_text='Brownian Walker in 3D',
                         autosize=False,
                         width=600,
                         height=500,
                         scene=dict(xaxis_title="x_pos",
                                    yaxis_title="y_pos",
                                    zaxis_title="time")
                         )
    return fig_BM

#Función para el plot Correlated Random Walk
@pn.depends(n_steps, x_pos, y_pos, speed, cauchy)
def CRW(n_steps, x_pos, y_pos, speed, cauchy):
    random = CRW_2d(cauchy, n_steps, speed, x_pos, y_pos)
    fig_CRW = go.Figure()

    fig_CRW.add_trace(go.Scatter3d(
        x=random.x_pos,
        y=random.y_pos,
        z=np.linspace(0, 1, n_steps),
        marker=dict(size=2),
        line=dict(width=2),
        mode='lines',
        name=f'CRW {round(cauchy, 2)}',
        showlegend=True
    ))
    fig_CRW.update_layout(title_text='Correlated Random Walk in 3D',
                          autosize=False,
                          width=600,
                          height=500,
                          scene=dict(xaxis_title="x_pos",
                                    yaxis_title="y_pos",
                                    zaxis_title="time")
                          )
    return fig_CRW

#Función para el plot del Lévy Flight
@pn.depends(n_steps, cauchy, alpha, x_pos, y_pos)
def LF(n_steps, cauchy, alpha, x_pos, y_pos):
    flight = LF_2d(cauchy, beta=alpha, n_steps=n_steps, x_pos=x_pos, y_pos=y_pos)
    fig_LF = go.Figure()

    fig_LF.add_trace(go.Scatter3d(
        x=flight.x_pos,
        y=flight.y_pos,
        z=np.linspace(0, 1, n_steps),
        marker=dict(size=2),
        line=dict(width=2),
        mode='lines',
        name=f'Levy {round(cauchy, 2)} alpha{alpha}',
        showlegend=True
    ))
    fig_LF.update_layout(title_text='Levy Flights in 3D',
                         autosize=False,
                         width=600,
                         height=500,
                         scene=dict(xaxis_title="x_pos",
                                    yaxis_title="y_pos",
                                    zaxis_title="time")
                         )
    return fig_LF



@pn.depends(metric_type, option, n_steps, x_pos, y_pos, speed, cauchy, alpha)
def met(metric, opcion, n_steps, x_pos, y_pos, speed, cauchy, alpha):
  if metric == 'Path Length':
    if opcion == 'BM':
      bm_df_1 = BM_2d(n_steps,speed,x_pos,y_pos)
      PL = path_length(bm_df_1)

      fig_path_length = go.Figure()

      fig_path_length.add_trace(go.Scatter(
          x = np.arange(len(PL))+1,
          y = PL,
          marker = dict(size=25),
          line = dict(width=25),
          mode = 'lines',
          name = 'path_length',
          showlegend = True
      ))
      fig_path_length.update_layout(title_text='Path length',
                        autosize=False,
                        width = 600,
                        height = 500)

      return fig_path_length
    elif opcion == 'CRW':
      random = CRW(cauchy,n_steps, speed, x_pos, y_pos)
      PL = path_length(random)

      fig_path_length = go.Figure()

      fig_path_length.add_trace(go.Scatter(
          x = np.arange(len(PL))+1,
          y = PL,
          marker = dict(size=25),
          line = dict(width=25),
          mode = 'lines',
          name = 'path_length',
          showlegend = True
      ))
      fig_path_length.update_layout(title_text='Path length',
                        autosize=False,
                        width = 600,
                        height = 500)
      return fig_path_length
    elif opcion == 'LF':
      flight=LF_2d(cauchy, beta=alpha, n_steps=n_steps, x_pos=x_pos, y_pos = y_pos)
      PL = path_length(flight)

      fig_path_length = go.Figure()

      fig_path_length.add_trace(go.Scatter(
          x = np.arange(len(PL))+1,
          y = PL,
          marker = dict(size=25),
          line = dict(width=25),
          mode = 'lines',
          name = 'path_length',
          showlegend = True
      ))
      fig_path_length.update_layout(title_text='Path length',
                        autosize=False,
                        width = 600,
                        height = 500)
      return fig_path_length

  elif metric == 'MSD':
    if opcion == 'BM':
      bm_df_1 = BM_2d(n_steps,speed,x_pos,y_pos)
      MeanSD = MSD(bm_df_1)
      fig_msd = go.Figure()

      fig_msd.add_trace(go.Scatter(
          x = np.arange(len(MeanSD)),
          y = MeanSD,
          marker = dict(size=2),
          line = dict(width=1),
          mode = 'lines',
          name = 'MSD',
          showlegend = True
      ))
      fig_msd.update_layout(title_text='MSD',
                        autosize=False,
                        width = 600,
                        height = 500)
      return fig_msd
    elif opcion == 'CRW':
      random = CRW(cauchy,n_steps, speed, x_pos, y_pos)
      MeanSD = MSD(random)
      fig_msd = go.Figure()

      fig_msd.add_trace(go.Scatter(
          x = np.arange(len(MeanSD)),
          y = MeanSD,
          marker = dict(size=2),
          line = dict(width=1),
          mode = 'lines',
          name = 'MSD',
          showlegend = True
      ))
      fig_msd.update_layout(title_text='MSD',
                        autosize=False,
                        width = 600,
                        height = 500)
      return fig_msd
    elif opcion == 'LF':
      flight=LF_2d(cauchy, beta=alpha, n_steps=n_steps, x_pos=x_pos, y_pos = y_pos)
      MeanSD = MSD(flight)
      fig_msd = go.Figure()

      fig_msd.add_trace(go.Scatter(
          x = np.arange(len(MeanSD)),
          y = MeanSD,
          marker = dict(size=2),
          line = dict(width=1),
          mode = 'lines',
          name = 'MSD',
          showlegend = True
      ))
      fig_msd.update_layout(title_text='MSD',
                        autosize=False,
                        width = 600,
                        height = 500)
      return fig_msd


###############################################################################################
# MOSTRAR DASHBOARD
###############################################################################################

pn.Row(pn.Column(markdown_text, option, opc), plot_trajectory, met, styles={'background': 'lightblue'})

Row(styles={'background': '...})
    [0] Column
        [0] Markdown(str)
        [1] RadioButtonGroup(name='option', options=['BM', 'CRW', 'LF'], value='BM', width=200)
        [2] ParamFunction(function, _pane=Column, defer_load=False)
    [1] ParamFunction(function, _pane=ParamFunction, defer_load=False)
    [2] ParamFunction(function, _pane=Plotly, defer_load=False)

2024-03-06 06:15:02,435 ERROR: panel.reactive - Callback failed for object named "Y position" changing property {'value': 2} 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 385, in _process_events
    self.param.update(**self_events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2278, in update
    restore = self_._update(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2318, in _update
    self_._batch_call_watchers()
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2509, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2471, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 865, in _replace_pane
    new_object = self.eval(self.object)
  File "/usr/local

2024-03-06 06:15:07,028 ERROR: panel.reactive - Callback failed for object named "option" changing property {'value': 'CRW'} 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 385, in _process_events
    self.param.update(**self_events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2278, in update
    restore = self_._update(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2318, in _update
    self_._batch_call_watchers()
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2509, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2471, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 865, in _replace_pane
    new_object = self.eval(self.object)
  File "/usr/local

2024-03-06 06:15:38,035 ERROR: panel.reactive - Callback failed for object named "option" changing property {'value': 'CRW'} 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 385, in _process_events
    self.param.update(**self_events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2278, in update
    restore = self_._update(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2318, in _update
    self_._batch_call_watchers()
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2509, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2471, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 865, in _replace_pane
    new_object = self.eval(self.object)
  File "/usr/local